Deep learning with PyTorch (A Reivew)

## 01. Import libraries

### Features of Pytorch

1. Simple interface:
2. Tools and libraries
3. Tensor Computation
4. Dynamic Graph computation
5. Variables
5. Parameters encirlce variables
6. MOdules
7. functions


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch.autograd import Variable

## 2 Building a network

In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes) -> None:
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [ ]:
# initialize our model with random data

model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

torch.Size([64, 10])


## 3. Set Devices

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 4. Hyper-parameters

In [ ]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1


In [ ]:
# Load data

train_dataset = datasets.MNIST(root = 'dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root = 'dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)



100%|██████████| 9912422/9912422 [00:00<00:00, 389555602.61it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 27917882.88it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 178991961.19it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21747178.96it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [ ]:
## Initialize the network
model = NN(input_size=input_size, num_classes=num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)



In [ ]:
# Train Network

for epoch in range(num_epochs):
   for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device)
    target = targets.to(device=device)

    # get correct shape (unroll to long vector
    data = data.reshape(data.shape[0], -1)

    # forward
    scores = model(data)
    loss = criterion(scores, target)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()



# check accuracy

def check_accuracy(loader, model):
  if loader.dataset.train:
    print("checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device= device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/ float(num_samples)*100:.2f}')

  model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



checking accuracy on training data
Got 55950 / 60000 with accuracy 93.25
Checking accuracy on test data
Got 9290 / 10000 with accuracy 92.90


In [ ]:
t = torch.tensor([1,2,3])

t

tensor([1, 2, 3])

In [ ]:
t = t.cuda()
t

tensor([1, 2, 3], device='cuda:0')

## Check GPU

In [ ]:
import torch

# if there's a GPU available...
if torch.cuda.is_available():

  # Tell Pytorch to use the GPU.
  device = torch.device('cuda')

  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  print('No GPU available, using th CPU instead')
  device = torch.device('CPU')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
